In [1]:
import numpy as np
#import hrr_cobot_robot
import spatialmath as sm
import rospy
np.set_printoptions(precision=5, suppress=True)

In [2]:
# #This is usually unnecessary
# def get_hostname_and_IP():
#     import socket
#     with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
#         s.connect(("8.8.8.8", 80))
#         return socket.gethostname(), s.getsockname()[0]

# _host, _ip = get_hostname_and_IP()
# %env ROS_MASTER_URI=http://127.0.0.1:11311
# # %env ROS_MASTER_URI=http://hrrcobotLinux54:11311
# print(f"\t\033[1m\033[4m!!!please check that the IPs below is identical!!!\033[0m"+
#       f"\n\thost {_host} has IP {_ip}, which should be identical to",end="\nvs:\t")
# %env ROS_IP=$_ip

In [3]:
rospy.init_node("NotebookDev_Changepositions2")

In [4]:
import hrr_cobot_robot as hrr_rob
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")

[rosout] [/NotebookDev_Changepositions2] Initialize MATLAB-Motion planner @ /home/hrr_cobot/_ros/hr_recycler_ws/src/planner/src
[rosout] [/NotebookDev_Changepositions2] connected to Matlab
The value of the ROS_IP environment variable, 192.168.2.87, will be used to set the advertised address for the ROS node.
Initializing global node /matlab_global_node_32420 with NodeURI http://192.168.2.87:45077/
[rosout] [/NotebookDev_Changepositions2] MATLAB-Initialization done


pybullet build time: May  8 2021 05:48:13


[rosout] some functions are not yet compiled. Expect delays upon first call


In [5]:
print(cobot)

Hrr-Cobot state:
FT-data:
current force:	[ 24.74396  -8.16138 -24.81152] [N]
current torque:	[-24.79828  21.7511    3.42749] [Nm]
current wrench:	[ 24.70422  -8.12325 -24.73078 -24.81956  21.75493   3.43898] [N,Nm]
=>in contact:	True
---
Robot-data:
q:		[ 145.9549    53.62703 -114.94745    0.11796  -79.84449   91.84506][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[-0.45842 -0.30993  0.21781][m]
quat:		0.34499 <[-0.31728 -0.6231   0.62615]>
tool-pos:	[-0.45842 -0.30993  0.21781][m]
tool-rpy:	[176.26097 -55.81803  87.73916][°]
robot-status:	ready
---


In [ ]:
cobot.change_tool("nothing")

In [ ]:
cobot.change_tool("screwdriver")

In [ ]:
cobot.T_B_E_robot, np.rad2deg(cobot.q)

In [ ]:
cobot.stop()

In [7]:
q_des = np.r_[ 1.34109,  0.09328, -1.44435, -0.01627,  0.67686, -0.00009]


In [7]:
cobot.q,q_des

(array([ 1.7797 ,  0.29935, -0.78164, -0.01453,  1.09394, -1.13418]),
 array([ 1.34109,  0.09328, -1.44435, -0.01627,  0.67686, -0.00009]))

In [8]:
cobot.move_to_joint_pose(q_des, stochastic=True)

> In HrRecyclerPerecptioPipeline (line 17)
> In HrRecyclerPerecptioPipeline (line 24)
> In gpmp2.test3Ddataset>add_obstacle (line 245)
In gpmp2.test3Ddataset (line 48)
In HrRecyclerPerecptioPipeline (line 51)

> In gpmp2.test3Ddataset>add_obstacle (line 245)
In gpmp2.test3Ddataset (line 49)
In HrRecyclerPerecptioPipeline (line 51)

calculating signed distance field ...
calculating signed distance field done
Elapsed time is 6.526431 seconds.
Elapsed time is 0.119776 seconds.


[rosout] [/NotebookDev_Changepositions2] Planner is done and execution is running ...


# Brief introduction

The tool change routine consists of two parts: pickup and disposal of a tool. For each tool, a sequence of positions for both pickup and dispose is recorded manually and saved in a python dictionary, with some additional information. Whenever a tool is changed with the actual action service, the corresponding dictionary is loaded and the robot moves through the positions in sequential order. The dictionaries look like this:
```
vacuum_2 = {
    "pickup": 
        {"poses_A": pickA, "poses_q": pickq,
        "close_tc": pickc, "use_joint_ctrl": pickj},
     "dispose":
        {"poses_A": dispA, "poses_q": dispq,
        "open_tc": dispo, "use_joint_ctrl": dispj}
    }
np.save("vacuum_2", vacuum_2)
```
Here the name vacuum_2 indicates that the dict concerns the vacuum gripper placed at the 2nd toolrack position.

vacuum2["pickup"] is a dict that contains several lists of equal length:

vacuum2["pickup"]["poses_A"]: A list of spatialmath SE3 objects with the pose of the robot

vacuum2["pickup"]["poses_q"]: A list of np.arrays with the corresponding joints of the robot

vacuum2["pickup"]["close_tc"]: A list of 0s and a single 1 indicating wether tool changer should close after arrival

vacuum2["pickup"]["use_joint_ctrl"]: A list of 0s and 1s indicating wether the robot should move linearly in joint space instead of in cartesian, in order to reach the position.

Here is an example of the pickup routine in action:
```
x = vacuum_2["pickup"]
for i in range(len(x["close_tc"])):
    if x["use_joint_ctrl"][i]:
        cobot.move_to_joint_pose(x["poses_q"][i])
    else:
        cobot.goTo(x["poses_A"][i],v_max=0.01)
    if x["close_tc"][i]:   
        #close tool changer

```
dispose routine is similiar, only "open_tc" instead of "close_tc".

At the beginning, the robot is in calibration pose with base joint turned so that the robot faces the begining of the toolrack.


# Record a fully new routine

## Record the pickup routine


### [Optional] Place robot in starting position - The first saved position should be reachable from here or anywhere above the tool rack without collision.

In [ ]:
cobot.stop()

In [ ]:
cobot.q_calib

In [ ]:
start=cobot.q_calib
start[0]=np.deg2rad(70)

In [ ]:
cobot.move_to_joint_pose(start)
#cobot.goTo(cobot.FK(start),v_max = 0.05)

In [15]:
cobot.close_tool_changer()

In [48]:
cobot.open_tool_changer()

### Initialize lists, move robot manually and append info to the lists at each important pose.

In [ ]:
print(cobot)

In [ ]:
cobot.T_B_E_robot

In [ ]:
pick_A = []
pick_q = []
pick_close_tc = []
pick_joint_ctrl = []

In [ ]:
pick_A

In [ ]:
# Run this cell for each position you want saved in the sequence
# have the calibration pose as first pose
## first position will be reached via motion planner and last one: motion planner works again, 
#then will go to calibration pose
pick_A.append(cobot.T_B_E_robot)
pick_q.append(cobot.q)
pick_close_tc.append(0) #Set to 1 if the current pose is the one where tool changer is closed
pick_joint_ctrl.append(0) #Set to 1 if the current pose was reached by joint control
#pick_A

In [ ]:
pick_A[0].t[2]=pick_A[0].t[2]-0.03


In [ ]:
pick_joint_ctrl.append(pick_joint_ctrl[0])

In [ ]:
pick_joint_ctrl.insert(-3, 0)

In [ ]:
pick_close_tc[7] = 0

In [ ]:
pick_close_tc

In [ ]:
pick_A.append(pick_A_last)

In [ ]:
pick_close_tc.insert(0, pick_close_tc[-1])

In [ ]:
pick_joint_ctrl.insert(0, pick_joint_ctrl[-1])

In [ ]:
pick_A[-3], pick_A[-4] = pick_A[-4], pick_A[-3]

In [ ]:
a = sm.SE3()
a.A[:4,:4] = [[-0.5543 , -0.0128 , -0.83222, -0.59505],
       [ 0.83121, -0.0602 , -0.55269, -0.1921],
       [-0.04302, -0.9981 ,  0.04401,  0.2394],
       [ 0.     ,  0.     ,  0.     ,  1.     ]]

In [ ]:
pick_A = 

In [ ]:
a

In [ ]:
pick_A = np.load("screwdriver_6_final_4.npy",allow_pickle=True).item()
pick_A

In [ ]:
-0.1911 - 0.001

In [ ]:
pick_A_star[3] = pick_A[0]

In [ ]:
pick_q_star[3] = pick_q[0]

In [ ]:
pick_A_star = x["pickup"]["poses_A"]


In [ ]:
pick_q_star = x["pickup"]["poses_q"]
#2.8757 ,  1.01699, -1.76747,  0.33417, -1.27525,  1.41246


In [ ]:
cobot.q

In [ ]:
pick_close_tc_star = x["pickup"]["close_tc"]


In [ ]:
pick_joint_ctrl_star= x["pickup"]["use_joint_ctrl"]


In [ ]:
pick_close_tc_star[3] = pick_close_tc[0]

In [ ]:
pick_joint_ctrl_star[3] =pick_joint_ctrl[0]

In [ ]:
pick_joint_ctrl_star

In [ ]:
x["pickup"]

Make sure that the last position is safe such that the robot can go to calibration pose without collision!

## Disposal

In [ ]:
start=cobot.q_calib
start[0]=np.deg2rad(70)
#cobot.move_to_joint_pose(start)
cobot.goTo(cobot.FK(start),v_max = 0.05)

In [ ]:
disp_A = []
disp_q = []
disp_open_tc = []
disp_joint_ctrl = []

In [ ]:
cobot.update_tf()
disp_A.append(cobot.T_B_E_robot)
disp_q.append(cobot.q)
disp_open_tc.append(0) #Set to 1 if the current pose is the one where tool changer is opened
disp_joint_ctrl.append(0) #Set to 1 if the current pose was reached by joint control
disp_A

In [ ]:
disp_A = [disp_A[0],disp_A[2],disp_A[1],disp_A[2],disp_A[3]]
disp_q = [disp_q[0],disp_q[2],disp_q[1],disp_q[2],disp_q[3]]
disp_open_tc = [0,0,1,0,0]
disp_joint_ctrl = [disp_joint_ctrl[0],disp_joint_ctrl[2],disp_joint_ctrl[1],disp_joint_ctrl[2],disp_joint_ctrl[3]]

In [ ]:
disp_joint_ctrl

In [ ]:
disp_joint_ctrl.append(disp_joint_ctrl[1])

In [ ]:
disp_joint_ctrl.append(temp)

In [ ]:
disp_A

In [ ]:
disp_joint_ctrl[-1]=1

In [ ]:
disp_open_tc[1] = 1

In [ ]:
disp_open_tc

In [ ]:
np.save("nextCutting_q", disp_q, allow_pickle=True)

Make sure that the last position is safe such that the robot can go to calibration pose or
anywhere above the toolracks without collision


## Save the routine
Possibly change the name in the last line from "vacuum_2" to shaftgrinder_1, wsg_3, ... see directory or below. Be careful with overwriting!

In [ ]:
x = {
    "pickup": 
        {"poses_A": pick_A, "poses_q": pick_q,
        "close_tc": pick_close_tc, "use_joint_ctrl": pick_joint_ctrl},
     "dispose":
        {"poses_A": list(reversed(pick_A)), "poses_q": list(reversed(pick_q)),
        "open_tc": list(reversed(pick_close_tc)), "use_joint_ctrl": list(reversed(pick_joint_ctrl))}
        
    }
np.save("wsg_3_final.npy", x, allow_pickle=True)

In [17]:
x["dispose"], x["pickup"]

({'poses_A': [   0.6174   -0.7328   -0.2859   -0.3993    
  -0.7795   -0.5214   -0.3471   -0.4496    
   0.1053    0.4372   -0.8932    0.7214    
   0         0         0         1         
,
      0.5605   -0.8281    0.01202  -0.4656    
  -0.8281   -0.5606   -0.00861  -0.521     
   0.01387  -0.005131 -0.9999    0.5417    
   0         0         0         1         
,
      0.5605   -0.8281    0.01198  -0.4656    
  -0.8281   -0.5606   -0.00865  -0.521     
   0.01388  -0.005075 -0.9999    0.3689    
   0         0         0         1         
,
      0.5605   -0.8281    0.01202  -0.4656    
  -0.8281   -0.5606   -0.008617 -0.521     
   0.01387  -0.005122 -0.9999    0.3478    
   0         0         0         1         
,
      0.5605   -0.8281    0.01198  -0.4656    
  -0.8281   -0.5606   -0.00865  -0.521     
   0.01388  -0.005075 -0.9999    0.3689    
   0         0         0         1         
,
      0.5605   -0.8281    0.01198  -0.4656    
  -0.8281   -0.5606   -0.008658 -0.52

In [15]:
x["dispose"]["open_tc"] = list(reversed(x["pickup"]["close_tc"]))

In [16]:
x["dispose"]["use_joint_ctrl"] = x["pickup"]["use_joint_ctrl"]
x["dispose"]["poses_q"]= list(reversed(x["pickup"]["poses_q"]))
x["dispose"]["poses_A"]= list(reversed(x["pickup"]["poses_A"]))

In [18]:
np.save("pgrip_4_final.npy", x, allow_pickle=True)

In [132]:
x["dispose"]["poses_A"], x["dispose"]["open_tc"]

([  -0.5614   -0.007518 -0.8275   -0.4109    
   0.8274    0.01351  -0.5614   -0.2787    
   0.0154   -0.9999   -0.001362  0.5877    
   0         0         0         1         
,
    -0.5614   -0.007513 -0.8275   -0.4109    
   0.8274    0.01352  -0.5614   -0.2787    
   0.0154   -0.9999   -0.001372  0.2399    
   0         0         0         1         
,
    -0.5614   -0.00751  -0.8275   -0.4674    
   0.8274    0.01352  -0.5614   -0.317     
   0.0154   -0.9999   -0.001375  0.2398    
   0         0         0         1         
,
    -0.5613   -0.007414 -0.8276   -0.4674    
   0.8275    0.01356  -0.5614   -0.317     
   0.01538  -0.9999   -0.001475  0.2218    
   0         0         0         1         
,
    -0.5613   -0.007422 -0.8276   -0.4572    
   0.8275    0.01355  -0.5614   -0.3101    
   0.01538  -0.9999   -0.001465  0.2218    
   0         0         0         1         
,
    -0.5594   -0.04657  -0.8276   -0.3902    
   0.8245    0.07123  -0.5614   -0.2647    
   0.08509

In [24]:
np.rad2deg(cobot.q)

array([ 76.83869,   5.34454, -82.75504,  -0.9322 ,  38.78117,  -0.00516])

In [104]:
x["dispose"]["use_joint_ctrl"]

[1, 1, 0, 0, 0, 0, 0, 0, 0, 1]

In [103]:
np.save("screwdriver_6_final.npy", x, allow_pickle=True)

In [ ]:
grasping contact y = -0.607

In [ ]:
x2 = x

In [ ]:
x2["pickup"]["poses_q"]

In [ ]:
x["pickup"]["poses_q"][-3] = np.deg2rad(x["pickup"]["poses_q"][-3])
x["pickup"]["poses_q"][-2] = np.deg2rad(x["pickup"]["poses_q"][-2])
x["pickup"]["poses_q"][-1] = np.deg2rad(x["pickup"]["poses_q"][-1])

In [ ]:
x["dispose"]["poses_q"][0] = np.deg2rad(x["dispose"]["poses_q"][0])
x["dispose"]["poses_q"][1] = np.deg2rad(x["dispose"]["poses_q"][1])
x["dispose"]["poses_q"][2] = np.deg2rad(x["dispose"]["poses_q"][2])

In [ ]:
x["pickup"]["poses_A"][2].t[1]=-0.607

In [ ]:
x["dispose"]["poses_A"]

# Load a routine to manipulate/test it

In [2]:
import numpy as np
import spatialmath as sm

In [11]:
#x = np.load("shaftgrinder_5_final.npy",allow_pickle=True).item()
#x = np.load("vacuum_2_final.npy",allow_pickle=True).item()
#x = np.load("wsg_3_final.npy",allow_pickle=True).item()
x = np.load("pgrip_4_final.npy",allow_pickle=True).item()
#x = np.load("screwdriver_6_final.npy",allow_pickle=True).item()

In [ ]:
q_des = np.r_[ 1.34109,  0.09328, -1.44435, -0.01627,  0.67686, -0.00009]


In [ ]:
cobot.move_to_joint_pose([ 1.34109,  0.09328, -1.44435, -0.01627,  0.67686, -0.00009], stochastic = True)

In [ ]:
x = np.load("screwdriver_6_final_4.npy",allow_pickle=True).item()

In [ ]:
x["pickup"]["close_tc"], x["pickup"]["close_tc"][3]

### Use this to change one entry

In [73]:
cobot.open_tool_changer()

In [78]:
cobot.close_tool_changer()

In [92]:
qq = cobot.q
AA = cobot.T_B_E_robot
qq,AA

(array([ 2.06855,  0.24089, -1.48786,  1.09183, -0.37386, -1.2097 ]),
    0.1353    0.9767   -0.1668   -0.2651    
  -0.04275  -0.1624   -0.9858   -0.5422    
  -0.9899    0.1405    0.01979   0.7146    
   0         0         0         1         
)

In [93]:
i = 9#entry
x["pickup"]["poses_A"][i]=AA
x["pickup"]["poses_q"][i]=qq
x["pickup"]["use_joint_ctrl"][i]=1
x["pickup"]["close_tc"][i]=0

In [97]:
x["pickup"]["use_joint_ctrl"][10]

1

In [23]:
i = 2#entry
x["pickup"]["poses_A"].append(AA)
x["pickup"]["poses_q"].append(qq)
x["pickup"]["use_joint_ctrl"].append(0)
x["pickup"]["close_tc"].append(0)

In [99]:
x["pickup"]["poses_A"].pop()
x["pickup"]["poses_q"].pop()
x["pickup"]["use_joint_ctrl"].pop()
x["pickup"]["close_tc"].pop()

0

In [77]:
x["pickup"]["close_tc"]

[0, 0, 0, 1, 0, 0, 0, 0, 0]

In [74]:
x["pickup"]["close_tc"][4]=0

In [12]:
cobot.close_tool_changer()

In [21]:
cobot.move_to_joint_pose(x["pickup"]["poses_q"][i], stochastic=False)

In [95]:
x["pickup"]["poses_q"], x["dispose"]["poses_q"]

([array([ 2.31032, -0.19496, -2.44117,  0.26769, -0.62876,  1.58366]),
  array([ 2.54561,  0.86011, -2.25274,  0.00006, -1.54061,  1.65599]),
  array([ 2.5456 ,  0.91938, -2.01526,  0.00005, -1.36238,  1.58617]),
  array([ 2.5456 ,  0.93133, -1.97696,  0.00005, -1.33602,  1.58617]),
  array([ 2.5456 ,  0.88422, -1.99433,  0.00008, -1.30638,  1.58618]),
  array([ 2.54561,  0.82088, -2.20106,  0.0001 , -1.44976,  1.58619]),
  array([ 2.54561,  0.01667, -2.07057,  0.0002 , -0.51509,  1.58603])],
 [array([ 2.54561,  0.01667, -2.07057,  0.0002 , -0.51509,  1.58603]),
  array([ 2.54561,  0.82088, -2.20106,  0.0001 , -1.44976,  1.58619]),
  array([ 2.5456 ,  0.88422, -1.99433,  0.00008, -1.30638,  1.58618]),
  array([ 2.5456 ,  0.93133, -1.97696,  0.00005, -1.33602,  1.58617]),
  array([ 2.5456 ,  0.91938, -2.01526,  0.00005, -1.36238,  1.58617]),
  array([ 2.54561,  0.86011, -2.25274,  0.00006, -1.54061,  1.65599]),
  array([ 2.31032, -0.19496, -2.44117,  0.26769, -0.62876,  1.58366])])

In [96]:
x["pickup"]["use_joint_ctrl"], x["dispose"]["use_joint_ctrl"]

([1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1])

In [97]:
x["dispose"]["use_joint_ctrl"][0] = 1

In [98]:
np.save("shaftgrinder_5_final.npy", x, allow_pickle=True)

In [107]:
x["pickup"]["close_tc"]

[0, 0, 0, 1, 0, 0, 0]

In [126]:
x["pickup"]["poses_A"].pop(4)
x["pickup"]["poses_q"].pop(4)
x["pickup"]["close_tc"].pop(4)
x["pickup"]["use_joint_ctrl"].pop(4)

0

In [108]:
a = [1,2,3,4,5,6,7]

In [109]:
a.insert(4,a[2])

In [124]:
a

[1, 2, 3, 4, 3, 5, 6, 7]

In [125]:
a.pop(2)

3

In [12]:
x["pickup"]

{'poses_A': [   0.6174   -0.7328   -0.2859   -0.3993    
  -0.7795   -0.5214   -0.3471   -0.4496    
   0.1053    0.4372   -0.8932    0.7214    
   0         0         0         1         
,
     0.5605   -0.8281    0.01198  -0.4656    
  -0.8281   -0.5606   -0.008658 -0.521     
   0.01388  -0.005065 -0.9999    0.5767    
   0         0         0         1         
,
     0.5605   -0.8281    0.01198  -0.4656    
  -0.8281   -0.5606   -0.00865  -0.521     
   0.01388  -0.005075 -0.9999    0.3689    
   0         0         0         1         
,
     0.5605   -0.8281    0.01202  -0.4656    
  -0.8281   -0.5606   -0.008617 -0.521     
   0.01387  -0.005122 -0.9999    0.3478    
   0         0         0         1         
,
     0.5605   -0.8281    0.01202  -0.4656    
  -0.8281   -0.5606   -0.00861  -0.521     
   0.01387  -0.005131 -0.9999    0.5417    
   0         0         0         1         
,
     0.6174   -0.7328   -0.2859   -0.3993    
  -0.7795   -0.5214   -0.3471   -0.4496    

In [7]:
x["pickup"]

{'poses_A': [   1         0.0005316  0.002363 -0.1719    
   0.0006258 -0.9992   -0.04004  -0.6194    
   0.00234   0.04005  -0.9992    0.6423    
   0         0         0         1         
,
     1         0.0003213  0.002358 -0.1719    
   0.0004156 -0.9992   -0.04008  -0.6194    
   0.002343  0.04008  -0.9992    0.3686    
   0         0         0         1         
,
     1         0.0003168  0.002372 -0.1719    
   0.0004116 -0.9992   -0.04004  -0.6194    
   0.002358  0.04004  -0.9992    0.3482    
   0         0         0         1         
,
     1         0.0003213  0.002358 -0.1719    
   0.0004156 -0.9992   -0.04008  -0.6194    
   0.002343  0.04008  -0.9992    0.3686    
   0         0         0         1         
,
     1         0.0005316  0.002363 -0.1719    
   0.0006258 -0.9992   -0.04004  -0.6194    
   0.00234   0.04005  -0.9992    0.6723    
   0         0         0         1         
],
 'poses_q': [array([ 1.84312,  0.60144, -0.88547, -0.01306,  1.61673, -1.29874

In [13]:
x["pickup"]["poses_A"].insert(4,x["pickup"]["poses_A"][2])
x["pickup"]["poses_q"].insert(4,x["pickup"]["poses_q"][2])
x["pickup"]["use_joint_ctrl"].insert(4,x["pickup"]["use_joint_ctrl"][2])
x["pickup"]["close_tc"].insert(4,x["pickup"]["close_tc"][2])

In [4]:
x["dispose"]["open_tc"]

[0, 1, 0, 0]

In [ ]:
x["dispose"]["poses_A"].insert(0,AA)
x["dispose"]["poses_q"].insert(0,qq)
x["dispose"]["use_joint_ctrl"].insert(0,0)
x["dispose"]["open_tc"].insert(0,0)
x["pickup"]["poses_A"].append(AA)
x["pickup"]["poses_q"].append(qq)
x["pickup"]["use_joint_ctrl"].append(1)
x["pickup"]["close_tc"].append(0)

In [ ]:
x["dispose"]["poses_A"].insert(0,disp_A[1])
x["dispose"]["poses_A"].insert(0,disp_A[0])
x["dispose"]["poses_q"].insert(0,disp_q[1])
x["dispose"]["poses_q"].insert(0,disp_q[0])
x["dispose"]["use_joint_ctrl"].insert(0,1)
x["dispose"]["use_joint_ctrl"].insert(0,0)
x["dispose"]["open_tc"].insert(0,0)
x["dispose"]["open_tc"].insert(0,0)

In [ ]:
x["pickup"]["poses_A"].append(disp_A[0])
x["pickup"]["poses_A"].append(disp_A[1])
x["pickup"]["poses_q"].append(disp_q[0])
x["pickup"]["poses_q"].append(disp_q[1])
x["pickup"]["use_joint_ctrl"].append(0)
x["pickup"]["use_joint_ctrl"].append(1)
x["pickup"]["close_tc"].append(0)
x["pickup"]["close_tc"].append(0)

In [ ]:
x["pickup"]["poses_q"].append(beforedispose_afterpickup)
x["pickup"]["poses_q"].append(totheleft)
x["pickup"]["poses_q"].append(after_before_unscrewing)

In [ ]:
x["dispose"]["use_joint_ctrl"].insert(0,1)

In [ ]:
x["dispose"]["open_tc"].insert(0,0)

In [ ]:
x["pickup"]["poses_A"].append(sm.SE3())

In [ ]:
#A small helper function to shift a pose's position by vec = [x y z] in m
def moveByVec(vec, pose):
    #shift position of se3 by vec
    return sm.SE3(vec)@pose

### Before automatic testing, check if automatic open/close tool changer works!

In [ ]:
cobot.close_tool_changer()#force=True)

In [ ]:
cobot.open_tool_changer()#force=True)

In [ ]:
cobot.stop()

In [ ]:
cobot.T_B_E_robot

### Pickup
Make sure that tool changer is open!

In [56]:
poses = x["pickup"]
#print(poses["close_tc"])

In [57]:
x["pickup"]["use_joint_ctrl"]

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]

In [52]:
cobot.open_tool_changer()

In [142]:
poses["close_tc"]

[0, 0, 0, 1, 0, 0, 0]

In [165]:
cobot.open_tool_changer(force=True)

In [60]:
#Manual loop
i = 2#run 0,1,2,3...
if poses["use_joint_ctrl"][i]:
    cobot.move_to_joint_pose(poses["poses_q"][i], stochastic=False) 
else:
    cobot.goTo(poses["poses_A"][i],v_max=0.01) 
if poses["close_tc"][i]:
    rospy.sleep(0.5)
    cobot.close_tool_changer(force=True)

[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are/is critical.
[rosout] joint(s) [1] are

In [ ]:
#Automatic runthrough
for i in range(len(poses["poses_A"])):
    if poses["use_joint_ctrl"][i]:
        rospy.sleep(0.5)
        cobot.move_to_joint_pose(poses["poses_q"][i], stochastic= False) 
    else:
        rospy.sleep(0.5)
        cobot.goTo(poses["poses_A"][i],v_max=0.01) 
    if poses["close_tc"][i]:
        rospy.sleep(0.5)
        cobot.close_tool_changer(force=True)

In [106]:
cobot.move_to_joint_pose(np.r_[ 1.34109,  0.09328, -1.44435, -0.01627,  0.67686, -0.00009], stochastic = False)

In [ ]:
cobot.stop()

### Dispose

In [107]:
poses = x["dispose"]

In [118]:
#Manual loop
i = 10
#change to 1,2,3...
if poses["use_joint_ctrl"][i]:
    cobot.move_to_joint_pose(poses["poses_q"][i],stochastic = False) 
else:
    cobot.goTo(poses["poses_A"][i],v_max=0.01) 
if poses["open_tc"][i]:
    rospy.sleep(0.5)
    cobot.open_tool_changer(force=True)

IndexError: list index out of range

In [33]:
cobot.FT_F

array([ 35.50507,   8.82202, -35.5957 , -38.34503,  37.79205,   1.88342])

In [ ]:
#Automatic runthrough
for i in range(len(poses["poses_A"])):
    if poses["use_joint_ctrl"][i]:
        rospy.sleep(0.5)
        cobot.move_to_joint_pose(poses["poses_q"][i], stochastic = False) 
    else:
        rospy.sleep(0.5)
        cobot.goTo(poses["poses_A"][i],v_max=0.01) 
    if poses["open_tc"][i]:
        rospy.sleep(0.5)
        cobot.open_tool_changer(force=True)

In [40]:
cobot.move_to_joint_pose(np.r_[1.34109, 0.09328, -1.44435, -0.01627, 0.67686, -0.00009], stochastic=False)

In [ ]:
twoCutting_q = np.load("twoCutting_q.npy")

In [ ]:
nextCutting_q = np.load("nextCutting_q.npy")

In [ ]:
twoCutting_q[0]

In [ ]:
cobot.move_to_joint_pose(twoCutting_q[0]) 

In [ ]:
cobot.move_to_joint_pose(twoCutting_q[1]) 

In [ ]:
cobot.move_to_joint_pose(twoCutting_q[1]) 